In [1]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.model import (
    QuantDenseModel,
    IntQuantDenseModel,
    IntQuantDenseModelLarge,
)
from utils.train import (
    train_for_epoch,
    test_model,
)
from utils.dataset import (
    get_smartpixel_dataloaders,
    get_global_smartpixel_dataloaders,   
)

DATASET_PATH = "./data/ds8_only/dec6_ds8_quant"  # Change me
EXPORT_PATH = "./model_outputs"

In [2]:
# Training parameters
learning_rate = 0.01
momentum = 0.5

# ++++ Quantization method (comment/uncomment) ++++
# Int quant
model = IntQuantDenseModel(in_features=13, dense_width=16, logit_total_bits=3, activation_total_bits=6, num_classes=12) # Small model

# model = IntQuantDenseModel(in_features=13, dense_width=58, logit_total_bits=4, activation_total_bits=8, num_classes=12) # Medium model

# model = IntQuantDenseModel(in_features=13, dense_width=512, logit_total_bits=4, activation_total_bits=8, num_classes=12) # Large model2

# model = IntQuantDenseModelLarge(in_features=13, dense_width=32, logit_total_bits=8, activation_total_bits=10, num_classes=12) # Large model

# Float quant
# model = FloatQuantLinearModel(
#     input_exponent_bit_width=2,
#     input_mantissa_bit_width=1,
#     weight_exponent_bit_width=4,
#     weight_mantissa_bit_width=3,
# )

# ---- Quantization method ----

device = (
    "xpu"
    if torch.xpu.is_available()  # For the only person who has an Intel GPU
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [3]:
train_loader, test_loader = get_global_smartpixel_dataloaders(
    DATASET_PATH, pin_memory=True, pin_memory_device=device
)

Merged 12 local datasets into global loaders.
  Training samples: 652160
  Test samples: 163040


In [4]:
for epoch in range(10):
    print(f"Epoch: {epoch}")
    train_for_epoch(model, device, train_loader, criterion, optimizer)
    test_model(model, device, test_loader, F.cross_entropy)

Epoch: 0


  0%|          | 0/10190 [00:00<?, ?it/s]

/home/z.ling.865/smartpixel-brevitas/venv/lib/python3.10/site-packages/torch/_tensor.py:1645: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1939.)
  return super().rename(names)


  0%|          | 0/160 [00:00<?, ?it/s]

Epoch: 1


...

  0%|          | 0/10190 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

Epoch: 9


  0%|          | 0/10190 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

In [5]:
# Get all correctly classified images/inputs from test dataset
correct_inputs, correct_targets = test_model(
    model, device, test_loader, F.cross_entropy, return_correct=True
)

# Get quantized weights and inputs
model_weights, model_scale = model.quant_weight()
correct_inputs, input_scale = model.quant_input(correct_inputs)

  0%|          | 0/160 [00:00<?, ?it/s]

In [6]:
MODEL_NAME = "small_int_quant"  # Change me for different models
EXPORT_PATH = os.path.join(EXPORT_PATH, MODEL_NAME)

# Create output directory if it doesn't exist
if not os.path.exists(EXPORT_PATH):
    os.makedirs(EXPORT_PATH)

# Export NumPy for compatibility with Rust
if isinstance(model_weights, dict):
    print("Exporting multi-layer quantized model weights...")
    for layer_name, weights in model_weights.items():
        np.save(EXPORT_PATH + f"{layer_name}_weights.npy", weights.cpu().numpy())

    for layer_name, scale in model_scale.items():
        np.save(EXPORT_PATH + f"{layer_name}_scale.npy", scale.cpu().numpy()) 
else:
    print("Exporting single-layer model weights...")
    np.save(EXPORT_PATH + "/model_weights.npy", model_weights.cpu().numpy())
    np.save(EXPORT_PATH + "/model_scale.npy", model_scale.cpu().numpy())
np.save(EXPORT_PATH + "/correct_inputs.npy", correct_inputs.cpu().numpy())
np.save(EXPORT_PATH + "/input_scale.npy", input_scale.cpu().numpy())
np.save(EXPORT_PATH + "/correct_targets.npy", correct_targets.cpu().numpy())

print(f"Exported model '{MODEL_NAME}' to {EXPORT_PATH}")

Exporting single-layer model weights...
Exported model 'small_int_quant' to ./model_outputs/small_int_quant


In [7]:
torch.save(model.state_dict(), EXPORT_PATH + "/smartpixel_model.pt")